In [1]:
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
import re
import json
from lxml import html

df = pd.read_csv('data_linkjob_test.csv')
# get links
links = df['link']
# create field 
df['job_name'] = ''
df['id_company'] = ''
df['company_name'] = ''
df['job_salary'] = ''
df['job_experience'] = ''
df['job_level'] = ''
df['job_expired_date'] = ''
df['job_details'] = ''
df['job_required'] = ''

In [3]:
def get_job_name(soup):
    #i = 0
    div_tag = soup.find('div', {'class': 'job-desc'})

    job_name = div_tag.find('h1', {'class': 'title'}).get_text()

    return job_name

In [10]:
def get_job_id_company(soup):
    div_tag = soup.find('div', {'class': 'job-desc'})
    div_tag2 = div_tag.find('a', {'class': 'employer job-company-name'})
    if div_tag2:
        href = div_tag2['href']
        job_id_company = href.split('.')[-2]
    else: job_id_company = ''
    
    return job_id_company
    

In [12]:
def get_job_company_name(soup):
    div_tag = soup.find('div', {'class': 'job-desc'})
    a_tag = div_tag.find('a', {'class': 'employer job-company-name'})
    if a_tag:
        job_company = a_tag.get_text()
    else: job_company = ''
    
    return job_company

In [84]:
def get_job_salary(soup):
    i_tag = soup.find('i', {'class': 'fa fa-usd'})
    li_tag = i_tag.parent.parent
    salary = li_tag.find('p').get_text()
              
    return salary

In [102]:
def get_job_experience(soup):
    i_tag = soup.find('i', {'class': 'fa fa-briefcase'})
    if i_tag :
        li_tag = i_tag.parent.parent
        experience = li_tag.find('p').get_text().strip()
        clean = re.sub(r'\s+', ' ', experience)
        experience = clean
    else :experience = "Không yêu cầu kinh nghiệm"
    
    return experience

In [165]:
def get_job_level(soup):
    i_tag = soup.find('i', {'class': 'mdi mdi-account'})
    if i_tag :
        li_tag = i_tag.parent.parent
        level = li_tag.find('p').get_text().strip()
    else :level = None
    
    return level

In [109]:
def get_job_expired_date(soup):
    i_tag = soup.find('i', {'class': 'mdi mdi-calendar-check'})
    if i_tag :
        li_tag = i_tag.parent.parent
        expired_date = li_tag.find('p').get_text().strip()
    else :expired_date = None
    
    return expired_date

In [160]:
def get_job_details(soup):
    div_tag = soup.find('div', {'class': 'detail-row reset-bullet'})
    # p_tags = div_tag.find_all('p')
    # job_details = ""
    # for p_tag in p_tags :
    #     job_details = job_details + "/n" + p_tag.get_text()
    job_details = div_tag.get_text(separator='\n', strip=True)
    
    return job_details

In [161]:
def get_job_required(soup):
    selected_element = soup.select_one('#tab-1 > section > div:nth-child(4)')
    job_required = selected_element.get_text(separator='\n', strip=True)

    return job_required

In [162]:
# Khởi tạo trình duyệt Edge
driver = webdriver.Edge()
i = 0
# Duyệt qua từng link và lấy thông tin
for link in links:
    # Mở trang web
    driver.get(link)

    # Chờ cho trang web tải xong
    driver.implicitly_wait(10)

    # Lấy HTML của trang web
    html = driver.page_source

    # Phân tích HTML bằng BeautifulSoup
    soup = BeautifulSoup(html, 'html.parser')

    #find job name
    job_name = get_job_name(soup)
    df['job_name'][i] = job_name
    
    #find company id
    id_company = get_job_id_company(soup)
    df['id_company'][i] = id_company
    
    #find company name
    company_name = get_job_company_name(soup)
    df['company_name'][i] = company_name
    
    #find job salary 
    job_salary = get_job_salary(soup)
    df['job_salary'][i] = job_salary
    
    #find job experience
    job_experience = get_job_experience(soup)
    df['job_experience'][i] = job_experience
    
    #find job level
    job_level = get_job_level(soup)
    df['job_level'][i] = job_level
    
    #find job expired date
    job_expired_date = get_job_expired_date(soup)
    df['job_expired_date'][i] = job_expired_date
    
    #find job details
    job_details = get_job_details(soup)
    df['job_details'][i] = job_details
    
    #find job required information
    job_required = get_job_required(soup)
    df['job_required'][i] = job_required
    
    print(str(i)+ " done")
    i = i + 1
    df.to_csv('data_no_duplicates.csv', index=False)

# Đóng trình duyệt
driver.quit()


0 done
1 done


# Convert CSV to JSON

In [164]:
# Đọc dữ liệu từ file CSV hoặc thực hiện các bước để có DataFrame của bạn
dfr = pd.read_csv('data_no_duplicates.csv')
# Chuyển đổi DataFrame thành danh sách bản ghi
records = dfr.to_dict(orient='records')

# Chuyển đổi dữ liệu ngày thành định dạng chuẩn (ví dụ: ISO 8601)
for record in records:
    if 'job_expired_date' in record and pd.notna(record['job_expired_date']):
        record['job_expired_date'] = pd.to_datetime(record['job_expired_date']).isoformat()
    if 'job_experience' in record and pd.isna(record['job_experience']):
        record['job_experience'] = None

# Ghi vào file JSON mà không sử dụng Unicode Escape
with open('job_data.json', 'w', encoding='utf-8') as json_file:
    json.dump(records, json_file, ensure_ascii=False, indent=2)
# Lưu DataFrame thành file JSON
#dfr.to_json('test.json', orient='records', lines=True)